# सरल इमेज क्लासिफायर

यह नोटबुक आपको एक प्री-ट्रेंड न्यूरल नेटवर्क का उपयोग करके इमेज को वर्गीकृत करना सिखाती है।

**आप क्या सीखेंगे:**
- प्री-ट्रेंड मॉडल को लोड और उपयोग कैसे करें
- इमेज प्रीप्रोसेसिंग
- इमेज पर भविष्यवाणी करना
- कॉन्फिडेंस स्कोर को समझना

**उपयोग का मामला:** इमेज में वस्तुओं की पहचान करना (जैसे "बिल्ली", "कुत्ता", "कार" आदि)

---


## चरण 1: आवश्यक लाइब्रेरी आयात करें

आइए उन उपकरणों को आयात करें जिनकी हमें आवश्यकता है। अगर आप इनमें से कुछ को अभी पूरी तरह से नहीं समझते हैं तो चिंता न करें!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## चरण 2: प्री-ट्रेंड मॉडल लोड करें

हम **MobileNetV2** का उपयोग करेंगे, जो पहले से ही लाखों छवियों पर प्रशिक्षित एक न्यूरल नेटवर्क है।

इसे **ट्रांसफर लर्निंग** कहा जाता है - किसी और द्वारा प्रशिक्षित मॉडल का उपयोग करना!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## चरण 3: सहायक फ़ंक्शन

आइए हमारे मॉडल के लिए छवियों को लोड और तैयार करने के लिए फ़ंक्शन बनाएं।


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## चरण 4: नमूना छवियों पर परीक्षण करें

चलो इंटरनेट से कुछ छवियों को वर्गीकृत करने की कोशिश करते हैं!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### प्रत्येक छवि को वर्गीकृत करें


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## चरण 5: अपनी खुद की छवियों को आज़माएं!

नीचे दिए गए URL को किसी भी छवि URL से बदलें जिसे आप वर्गीकृत करना चाहते हैं।


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 अभी क्या हुआ?

1. **हमने एक प्री-ट्रेंड मॉडल लोड किया** - MobileNetV2 को लाखों तस्वीरों पर प्रशिक्षित किया गया था  
2. **हमने तस्वीरों को प्रीप्रोसेस किया** - उन्हें मॉडल के लिए सही आकार और फॉर्मेट में बदला  
3. **मॉडल ने प्रेडिक्शन किया** - यह 1000 ऑब्जेक्ट क्लासेस के लिए संभावनाएं बताता है  
4. **हमने परिणामों को डिकोड किया** - अंकों को इंसानों के पढ़ने लायक लेबल में बदला  

### कॉन्फिडेंस स्कोर को समझना

- **90-100%**: बहुत ज्यादा भरोसेमंद (लगभग निश्चित रूप से सही)  
- **70-90%**: भरोसेमंद (शायद सही)  
- **50-70%**: थोड़ा भरोसेमंद (शायद सही हो सकता है)  
- **50% से कम**: ज्यादा भरोसेमंद नहीं (अनिश्चित)  

### प्रेडिक्शन गलत क्यों हो सकता है?

- **अजीब कोण या रोशनी** - मॉडल को सामान्य तस्वीरों पर प्रशिक्षित किया गया था  
- **कई ऑब्जेक्ट्स** - मॉडल एक मुख्य ऑब्जेक्ट की उम्मीद करता है  
- **दुर्लभ ऑब्जेक्ट्स** - मॉडल केवल 1000 कैटेगरी को जानता है  
- **कम गुणवत्ता वाली तस्वीर** - धुंधली या पिक्सेलेटेड तस्वीरें पहचानना मुश्किल होता है  

---  


## 🚀 अगले कदम

1. **अलग-अलग तस्वीरें आज़माएं:**
   - [Unsplash](https://unsplash.com) पर तस्वीरें खोजें
   - राइट-क्लिक करें → "Copy image address" से URL प्राप्त करें

2. **प्रयोग करें:**
   - अमूर्त कला के साथ क्या होता है?
   - क्या यह अलग-अलग कोणों से वस्तुओं को पहचान सकता है?
   - यह कई वस्तुओं को कैसे संभालता है?

3. **अधिक जानें:**
   - [Computer Vision पाठ](../lessons/4-ComputerVision/README.md) का अन्वेषण करें
   - अपना खुद का इमेज क्लासिफायर ट्रेन करना सीखें
   - समझें कि CNNs (Convolutional Neural Networks) कैसे काम करते हैं

---

## 🎉 बधाई हो!

आपने अभी-अभी एक अत्याधुनिक न्यूरल नेटवर्क का उपयोग करके एक इमेज क्लासिफायर बनाया है!

यही तकनीक शक्ति देती है:
- Google Photos (आपकी तस्वीरों को व्यवस्थित करना)
- सेल्फ-ड्राइविंग कारें (वस्तुओं को पहचानना)
- चिकित्सा निदान (एक्स-रे का विश्लेषण करना)
- गुणवत्ता नियंत्रण (त्रुटियों का पता लगाना)

खोज और सीखते रहें! 🚀



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
